In [1]:
import cv2
import numpy as np
import dlib
from imutils import face_utils

# Read Image
im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/KakaoTalk_20180215_172502292.jpg");
im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/KakaoTalk_20180215_172527978.jpg");
im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/20170108_195652 (2).jpg");
# im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/KakaoTalk_20180128_102715442 (2).jpg");
# im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/1525290878812.jpg");
# im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/1525290879707.jpg");
im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/1525290880382.jpg");
im = cv2.imread("C:/Users/ChungkiLee/Pictures/Saved Pictures/KakaoTalk_20180507_042030602.jpg");

size = im.shape

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:/Users/ChungkiLee/Documents/GitHub/webcam/previewimgs/shape_predictor_68_face_landmarks.dat')

gray  = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 0)
for rect in rects:
    shape0 = predictor(gray, rect)
    shape0 = np.array(face_utils.shape_to_np(shape0))

#2D image points. If you change the image, you need to change vector
image_points = np.array([
                            (shape0[33, :]),     # Nose tip
                            (shape0[8,  :]),     # Chin
                            (shape0[36, :]),     # Left eye left corner
                            (shape0[45, :]),     # Right eye right corne
                            (shape0[48, :]),     # Left Mouth corner
                            (shape0[54, :])      # Right mouth corner
                        ], dtype="double")
 
# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner                     
                        ])

# Camera internals
focal_length = size[1]
center = (size[1]/2, size[0]/2)
camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )
print ("Camera Matrix :\n {0}".format(camera_matrix))

dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
(success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)
print ("Rotation Vector:\n {0}".format(rotation_vector))
print ("Translation Vector:\n {0}".format(translation_vector))
 
# Project a 3D point (0, 0, 1000.0) onto the image plane.
# We use this to draw a line sticking out of the nose
(nose_end_point2D, jacobian) = cv2.projectPoints(np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs)
 
for p in image_points:
    cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)

p1 = ( int(image_points[0][0]), int(image_points[0][1]))
p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
 
cv2.line(im, p1, p2, (255,0,0), 2) 
# Display image
cv2.imshow("Output", im)
cv2.waitKey(0)

while(1):
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        cv2.destroyAllWindows()
        break


Camera Matrix :
 [[ 720.    0.  360.]
 [   0.  720.  640.]
 [   0.    0.    1.]]
Rotation Vector:
 [[-2.68507902]
 [ 0.24375015]
 [-0.71329851]]
Translation Vector:
 [[  277.03115486]
 [ -528.83141574]
 [ 2492.87714354]]
